# Dueling Deep Q-Learning for Lunar Landing

El aterrizaje lunar ha sido un desafío emocionante y emblemático en la historia de la exploración espacial. La capacidad de aterrizar un módulo lunar de manera segura y precisa en la superficie de la luna es fundamental para el éxito de las misiones espaciales. En un intento de abordar este desafío, el campo del aprendizaje por refuerzo ha surgido como un enfoque interesante para abordarlo.

En este proyecto, nos adentramos en el mundo del aprendizaje por refuerzo y nos enfrentamos al desafío del aterrizaje lunar utilizando Deep Q-Learning (DQN), una técnica que combina el aprendizaje profundo con métodos de aprendizaje por refuerzo. Nuestro objetivo es entrenar un agente de inteligencia artificial para aterrizar un módulo lunar en el entorno simulado de Lunar Lander de OpenAI Gym.

**LUNAR LANDER ENVIRONMENT**

Lunar Lander es un entorno clásico de aprendizaje por refuerzo que simula el desafío de aterrizar un módulo lunar en la superficie lunar. El agente controla la inclinación y la potencia del motor del módulo lunar para guiar su descenso y evitar colisiones. El objetivo es lograr un aterrizaje suave y seguro, maximizando al mismo tiempo la eficiencia del combustible.

**Enfoque Dueling DQN**

En lugar de utilizar el enfoque estándar de DQN, decidimos implementar Dueling DQN para abordar el desafío del aterrizaje lunar. Dueling DQN es una variante del algoritmo DQN que se centra en descomponer la función de valor Q en componentes de valor de estado y ventaja. Esto permite al agente aprender de manera más eficiente y separar la evaluación del estado de la selección de la acción.

En Dueling DQN, la arquitectura de la red neuronal se divide en dos ramas: una que estima el valor del estado y otra que estima la ventaja de cada acción. Estas estimaciones se combinan posteriormente para obtener la función de valor Q final. Esta descomposición permite al agente centrarse en aprender la importancia relativa de cada acción en lugar de intentar estimar directamente el valor de cada acción.

**Implementación y resultados**

En nuestro proyecto, adaptamos la arquitectura de la red neuronal para incorporar la lógica de Dueling DQN. Configuramos los hiperparámetros relevantes y diseñamos una estrategia de entrenamiento que aprovecha la estructura de la red neuronal dueling.

A lo largo de múltiples episodios de entrenamiento, observamos que nuestro agente entrenado con Dueling DQN logró mejorar su capacidad para aterrizar de manera precisa y eficiente en comparación con el enfoque estándar de DQN. La descomposición de la función de valor Q en componentes de estado y ventaja permitió al agente aprender de manera más efectiva y generalizar mejor en diferentes situaciones.

**Conclusiones y perspectivas futuras**

Nuestro proyecto destaca la efectividad del enfoque Dueling DQN para mejorar el rendimiento de los agentes de inteligencia artificial en entornos desafiantes de aprendizaje por refuerzo, como el aterrizaje lunar. La capacidad de descomponer la función de valor Q en componentes separados proporciona una manera más eficiente de aprender y tomar decisiones.

En el futuro, planeamos explorar aún más las capacidades del enfoque Dueling DQN en otros entornos de simulación y aplicaciones del mundo real. Además, consideramos investigar técnicas adicionales de aprendizaje por refuerzo para seguir mejorando la capacidad y la robustez de nuestros agentes, con el objetivo final de aplicar estos avances en la exploración espacial y otros campos relacionados.







### Instalación e importación librerías

In [29]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [30]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple
import gymnasium as gym
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

### Construimos la red neuronal

#### Feed Neural Network

Esta red neuronal tiene una estructura similar a una red neuronal convencional, con dos capas completamente conectadas (fully connected) seguidas de activaciones ReLU. Sin embargo, la diferencia clave es en la forma en que calcula los valores Q. En lugar de calcular directamente los valores Q a partir de las capas finales, calcula los valores de ventaja y valor por separado y luego los combina para obtener los valores Q finales. Esto se hace para permitir que el agente diferencie entre la calidad intrínseca de un estado (valor) y la calidad de cada acción en ese estado (ventaja).

In [31]:
class DuelingNetwork(nn.Module):

  def __init__(self, state_size, action_size, seed=21):
    super(DuelingNetwork, self).__init__()
    self.seed = torch.manual_seed(seed)
    self.fc1 = nn.Linear(state_size, 64)
    self.fc2 = nn.Linear(64, 64)


    self.fc_value = nn.Linear(64, 1)
    self.fc_advantage = nn.Linear(64, action_size)

  def forward(self, state):
    x = F.relu(self.fc1(state))
    x = F.relu(self.fc2(x))

    value = self.fc_value(x)
    advantage = self.fc_advantage(x)


    q_values = value + (advantage - advantage.mean(dim=1, keepdim=True))

    return q_values

### Configuración del entorno Lunar Lander

En esta parte del código, se inicializa el entorno del Lunar Lander utilizando la biblioteca Gym de OpenAI.

In [32]:
env = gym.make('LunarLander-v2')
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (8,)
State size:  8
Number of actions:  4


Importamos la biblioteca Gym, que proporciona una variedad de entornos para experimentar con algoritmos de aprendizaje por refuerzo. Creamos una instancia del entorno Lunar Lander utilizando la función make() de Gym. 'LunarLander-v2' es el identificador del entorno, que indica que queremos usar la versión 2 del entorno Lunar Lander. Esta versión del entorno es una versión actualizada y mejorada del entorno original.
Lunar Lander es un entorno clásico de aprendizaje por refuerzo donde el objetivo del agente es aterrizar un módulo lunar de forma segura en la superficie lunar, evitando colisiones y consumiendo la menor cantidad de combustible posible.

Obtenemos la forma del espacio de observación del entorno Lunar Lander (state shape), que describe la forma de los estados que el agente recibe como entrada. En este caso, el espacio de observación consiste en un vector con 8 dimensiones.

Obtenemops el tamaño del espacio de observación del entorno Lunar Lander, que es igual a la dimensión del espacio de observación. En este caso, state_size representa la cantidad de variables de estado del entorno. El estado del entorno se define por un vector de 8 dimensiones que contiene información sobre la posición, velocidad y orientación del módulo lunar, así como información sobre las piernas del módulo lunar (si están en contacto con la superficie lunar o no). Esta información permite al agente tomar decisiones informadas sobre cómo controlar el módulo lunar.

Obtenemos el número de acciones posibles que el agente puede tomar en el entorno Lunar Lander con la función 'env.action_space.n'. Esta devuelve la cantidad de acciones en el espacio de acciones del entorno. El agente puede elegir entre 4 acciones discretas en cada paso de tiempo:

- No hacer nada
- Encender el motor principal
- Encender el motor izquierdo
- Encender el motor derecho




### Inicialización Hiperparámetros

En esta parte del notebook decidimos definir los hiperparmétros

In [33]:
learning_rate = 5e-4
minibatch_size = 100
discount_factor = 0.99
replay_buffer_size = int(1e5)
interpolation_parameter = 1e-3

- Tasa de aprendizaje (learning_rate): La tasa de aprendizaje controla la magnitud de los ajustes que se realizan a los pesos de la red neuronal durante el entrenamiento.

- Tamaño del minibatch (minibatch_size): El tamaño del minibatch determina cuántas muestras de experiencia se utilizan en cada paso de entrenamiento de la red neuronal.

- Factor de descuento (discount_factor): El factor de descuento controla la importancia relativa de las recompensas futuras en comparación con las recompensas inmediatas. Un factor de descuento cercano a 1 indica que el agente valora altamente las recompensas futuras, lo que fomenta la planificación a largo plazo. Un factor de descuento cercano a 0 indica que el agente valora menos las recompensas futuras.
- Tamaño del búfer de repetición (replay_buffer_size): El tamaño del búfer de repetición determina cuántas experiencias pasadas se almacenarán para el entrenamiento de la red neuronal. El búfer de repetición se utiliza en el método de memoria de repetición para almacenar y muestrear experiencias pasadas de manera eficiente.

- Parámetro de interpolación (interpolation_parameter): El parámetro de interpolación se utiliza en el proceso de actualización suave de los pesos de la red neuronal objetivo. La actualización suave es una técnica que suaviza los cambios en los pesos de la red objetivo al combinar gradualmente los pesos de la red local y los pesos de la red objetivo. El parámetro de interpolación controla la proporción de contribución de los pesos de la red local y los pesos de la red objetivo en la actualización suave.


### Implementación Experience Replay

En esta parte del código definimos la memoria de repetición, que es una técnica importante en el aprendizaje por refuerzo.

In [34]:
class ReplayMemory(object):

  def __init__(self, capacity):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity
    self.memory = []

  def push(self, event):
    self.memory.append(event)
    if len(self.memory) > self.capacity:
      del self.memory[0]

  def sample(self, batch_size):
    experiences = random.sample(self.memory, k = batch_size)
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    return states, next_states, actions, rewards, dones

El constructor inicializa la memoria de repetición con una capacidad máxima especificada y el número máximo de experiencias pasadas que la memoria puede almacenar.

Agregamos el método push para añadir una nueva experiencia al búfer de repetición.En el que creamos la tupla event que contiene la experiencia del agente.
Si la memoria ya ha alcanzado su capacidad máxima, se elimina la experiencia más antigua para dejar espacio para la nueva experiencia.

También definimos el  método sample para realizar un muestreo aleatorio de un lote de experiencias del búfer de repetición.
batch_size especifica el tamaño del lote, es decir, cuántas experiencias se deben muestrear.
Se utilizan las experiencias muestreadas para construir tensores de estados, acciones, recompensas, siguientes estados y señales de terminación, que se utilizan para el entrenamiento de la red neuronal.

### Implementación del Agente

#### Dueling DQN

Definimos el agente Dueling DQN:

In [35]:
class Dueling_DQN():

  def __init__(self, state_size, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size
    self.local_qnetwork = DuelingNetwork(state_size, action_size).to(self.device)
    self.target_qnetwork = DuelingNetwork(state_size, action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
    self.memory = ReplayMemory(replay_buffer_size)
    self.t_step = 0

  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))
    self.t_step = (self.t_step + 1) % 4
    if self.t_step == 0:
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor)

  def act(self, state, epsilon = 0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))
  def learn(self, experiences, discount_factor):
    states, next_states, actions, rewards, dones = experiences


    next_q_values = self.target_qnetwork(next_states).detach()
    next_q_values_max, _ = next_q_values.max(dim=1, keepdim=True)
    q_targets = rewards + (discount_factor * next_q_values_max * (1 - dones))
    q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)

    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()


    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

Esta clase define un agente que interactúa con el entorno Lunar Lander utilizando un método de aprendizaje por refuerzo conocido como Dueling DQN (Doble Red Neuronal Q con Duelo).

Inicializamos el agente mediante una clase. En ella, utilizamos el tamaño del espacio de estado y el tamaño del espacio de acción definidos previamente. Además, creamos un par de instancias de la clase Network que representan las redes neuronales que aproximan la función Q. Una red neuronal representa el valor Q para cada par estado-acción.
También definimos un optimizador Adam que se utiliza para actualizar los pesos de la red neuronal local durante el entrenamiento, y una instancia de ReplayMemory que almacena las experiencias pasadas del agente para su posterior uso en el entrenamiento. Finalmente, establecemos un contador que controla cuándo se actualizan los pesos de la red neuronal objetivo.

Luego, definimos el método 'step', que se llama cada vez que el agente realiza una acción en el entorno. Agrega la experiencia actual (estado, acción, recompensa, próximo estado, hecho) a la memoria de repetición. Incrementa self.t_step y, si self.t_step es un múltiplo de 4, se realiza un paso de aprendizaje llamando al método learn().
El método 'act' elige una acción para el agente dado un estado. Si epsilon es mayor que cero, se elige una acción de manera epsilon-greedy, lo que significa que el agente elige una acción aleatoria con probabilidad epsilon y la mejor acción según la red neuronal local con probabilidad 1 - epsilon. Si epsilon es cero, el agente siempre elige la mejor acción según la red neuronal local.
El método 'learn' realiza un paso de aprendizaje utilizando una muestra de experiencias pasadas de la memoria de repetición. Calcula los objetivos Q utilizando la red neuronal objetivo y la función de pérdida de error cuadrático medio (MSE) entre los valores Q predichos por la red neuronal local y los objetivos Q calculados. Actualiza los pesos de la red neuronal local utilizando el optimizador Adam y realiza una actualización suave de los pesos de la red neuronal objetivo.
El método 'soft_update' actualiza suavemente los pesos de la red neuronal objetivo utilizando una interpolación entre los pesos de la red neuronal local y los pesos de la red neuronal objetivo actuales. Esto ayuda a estabilizar el entrenamiento y mejorar el rendimiento del agente.







Instanciamos el agente DQN, utilizando el tamaño del espacio de estado y el número de acciones disponibles definidas previamente.

In [36]:
agent = Dueling_DQN(state_size, number_actions)

### Entrenamiento del agente

En este chunk podemos observar los parámetros necesarios para proceder con el entrenamiento del agente:

- `number_episodes`: Número total de episodios de entrenamiento que se ejecutarán.
- `maximum_number_timesteps_per_episode`: Número máximo de pasos de tiempo permitidos por episodio.
- `epsilon_starting_value`: Valor inicial de epsilon para la estrategia epsilon-greedy.
- `epsilon_ending_value`: Valor final de epsilon después del decaimiento.
- `epsilon_decay_value`: Factor de decaimiento para epsilon en cada episodio.
- `epsilon`: Inicialización de epsilon con su valor inicial.
- `scores_on_100_episodes`: Una cola de tamaño fijo que almacena las puntuaciones de los últimos 100 episodios para calcular el promedio móvil de la puntuación.

In [37]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value  = 1.0
epsilon_ending_value  = 0.01
epsilon_decay_value  = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)


Y ahora procedemos a detallar el proceso de entrenamiento del agente:

Se ejecuta un bucle que itera sobre cada episodio de entrenamiento.
En cada episodio, se reinicia el entorno (env.reset()) para obtener el estado inicial. Luego, se ejecuta un bucle interno que itera sobre cada paso de tiempo en el episodio actual.
En cada paso de tiempo, el agente selecciona una acción utilizando su política actual, que puede ser epsilon-greedy (agent.act()).
La acción se ejecuta en el entorno (env.step(action)) y se obtiene el siguiente estado, la recompensa, y si el episodio ha terminado.
El agente almacena la experiencia actual en su memoria de repetición (agent.step()).
Se actualiza el estado actual con el siguiente estado y se suma la recompensa al puntaje acumulado del episodio.
Si el episodio ha terminado (done == True), el ciclo interno se detiene.
Se agrega el puntaje del episodio a la cola de puntuaciones de los últimos 100 episodios.
Se actualiza el valor de epsilon utilizando el decaimiento programado.
Se imprime el número de episodio y el promedio móvil de la puntuación de los últimos 100 episodios.
Si el promedio móvil de la puntuación de los últimos 100 episodios alcanza 200 o más, se imprime un mensaje indicando que el entorno se ha resuelto y se guarda el modelo del agente.
El ciclo de entrenamiento se detiene si se resuelve el entorno o si se alcanza el número máximo de episodios de entrenamiento.

In [38]:

for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

Episode 100	Average Score: -181.79
Episode 200	Average Score: -129.62
Episode 300	Average Score: -90.43
Episode 400	Average Score: -4.07
Episode 500	Average Score: -8.86
Episode 600	Average Score: 6.29
Episode 700	Average Score: 57.39
Episode 800	Average Score: 192.95
Episode 815	Average Score: 200.37
Environment solved in 815 episodes!	Average Score: 200.37


### Visualización los resultados

Estas funciones se utilizan para visualizar un video del modelo entrenado en el entorno Lunar Lander v2. La primera función crea y guarda el video, mientras que la segunda función muestra el video si está disponible.

In [40]:
def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()